# Units and Constants

This notebook introduces some of the most commonly used functionality when working with PlasmaPy...

## Units

In scientific programming, it is common to represent physical quantities as numbers.

In [ ]:
distance_in_miles = 5
time_in_hours = 60
velocity_in_mph = distance_in_miles / time_in_hours

Representing physical quantities as numbers comes with some risks, such as accidentally performing operations with different units like `time_in_seconds + time_in_hours` or incompatible units like `length + time`.

To avoid situations like this, we can use `astropy.units`. It's standard to import this subpackage as `u`.

In [ ]:
import astropy.units as u

We can combine a number with a unit by multiplying them together.

In [ ]:
distance = 8 * u.km

The `distance` object that we created is a `Quantity` that combines a number (or array of numbers) with a physical unit.

In [ ]:
type(distance)

Operations between `Quantity` objects handle unit operations automatically.

In [ ]:
time = 60 * u.hr
velocity = distance / time
print(velocity)

We can convert a `Quantity` to different units using the `.to` method of a `Quantity`. This method accepts strings that represent a unit (including compound units) or a unit object.

In [ ]:
velocity.to("m/s")

In [ ]:
velocity.to(u.m / u.s)

The `.si` and `.cgs` attributes provide the `Quantity` in SI or CGS units, respectively.

In [ ]:
velocity.si

In [ ]:
velocity.cgs

We can add two `Quantity` objects together if they have different units, as long as they have compatible physical types.

In [ ]:
1 * u.m + 1 * u.cm

When we attempt operations with physically incompatible units, an exception is raised. This helps us prevent common errors.

In [ ]:
3 * u.m + 3 * u.s

We can create dimensionless `Quantity` objects too.

In [ ]:
3 * u.dimensionless_unscaled

We can use `astropy.units` to help create recipes with ridiculous units.

In [ ]:
volume = u.barn * u.Mpc
volume.to(u.imperial.tsp)

It is common for plasma physicists to use electron-volts (eV) as units of temperature, or rather as the temperature divided by the Boltzmann constant. When working with a units package, this can cause some problems.  To handle this, `astropy.units` has built-in *equivalencies*. 

In [ ]:
thermal_energy_per_particle = 1 * u.eV
thermal_energy_per_particle.to("K", equivalencies=u.temperature_energy())

## Contants

We can access most commonly needed physical constants from `astropy.constants`.

In [ ]:
from astropy import constants

In [ ]:
constants.c

In [ ]:
constants.k_B